- AIhub의 뉴스기사 문서요약 데이터의 '추출요약문' 및 '생성요약문'의 종결어미를 문어체에서 구어체로 변환합니다.
- 미흡한 점 : 앞선 자모를 고려해야 하는 '- 어/아' 는 세분화하지 못하고 '어'로 변환했음
    - 예시 ) 받는다. -> 받어요.

In [1]:
%%time

import os
import csv
import pandas as pd

file_name = os.getenv('HOME')+'/aiffel/aeumgil/AIhub/data/aihub_eda.csv'
df = pd.read_csv(file_name)

CPU times: user 14.7 s, sys: 1.72 s, total: 16.4 s
Wall time: 16 s


In [2]:
df.head()

,title,original_text,ext_sum,ext_sum_lst,summary,size,char_count,sum_char_cnt,ext_char_cnt,readable,accurate,informative,trustworthy,media_sub_type
0,"논 타작물 재배, 2월 말까지 신청하세요",전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...,['전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...,small,927,155,190,4,3,3,3,지역지
1,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성",여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...,여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...,['여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 ...,여수시는 컬러빌리지 사업에 8억원을 투입하여 색채와 빛 도시를 완성하여 고소천사벽화...,small,764,118,182,4,4,5,4,지역지
2,“새해 정기 받고 올해는 반드시 일내자!”,전남드래곤즈(사장 신승재)는 지난 4일 구봉산 해맞이 행사를 통해 새해 각오를 다졌...,임직원과 선수단 모두는 이날 구봉산 정상에 올라 일출을 보며 2018년 구단 목표를...,['임직원과 선수단 모두는 이날 구봉산 정상에 올라 일출을 보며 2018년 구단 목...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...,medium,1066,68,216,4,4,4,4,지역지
3,"농업인 역량 강화, 새해 실용교육 실시",광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 1050...,광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 1050...,['광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 10...,"광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작...",small,746,68,211,4,4,5,4,지역지
4,타이완 크루즈관광객 4천여명‘전남’온다,타이완의 크루즈관광객 4000여명이 올해 두 차례에 걸쳐 여수에 입항한다. 전라남도...,전라남도는 올해 4월과 6월 홍콩 크루즈선사 스타크루즈의 5만톤급 아쿠아리우스 호가...,['전라남도는 올해 4월과 6월 홍콩 크루즈선사 스타크루즈의 5만톤급 아쿠아리우스 ...,올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...,small,978,66,257,4,4,4,4,지역지


In [4]:
import ast
def eval_(x): return ast.literal_eval(x)

df['ext_sum_lst'] = df['ext_sum_lst'].apply(eval_)

In [5]:
from konlpy.tag import Mecab

mecab = Mecab()

In [6]:
#- 불규칙 종결어미 변환 사전
convert_dict = {
 '뒸다.':'뒀어요.',
 'DP.':'',
 '한다.':'해요.',
 '보인다.':'보여요.',
 '된다.':'돼요.',
 '열린다.':'열려요.',
 '나선다.':'나서요.',
 '선보인다.':'선보여요.',
 '연다.':'열어요.',
 '나온다.':'나와요.',
 '.':'.',
 '보여진다.':'보여져요.',
 '펼친다.':'펼쳐요.',
 '펼쳐진다.':'펼쳐져요.',
 '들어간다.':'들어가요.',
 '준다.':'줘요.',
 '끈다.':'끌어요.',
 '보여준다.':'보여줘요.',
 '바란다.':'바라요.',
 '가진다.':'가져요.',
 '주어진다.':'주어져요.',
 '본다.':'봐요.',
 '쏠린다.':'쏠려요.',
 '치른다.':'치러요.',
 '벌인다.':'벌여요.',
 '꼽힌다.':'꼽혀요.',
 '자아낸다.':'자아내요.',
 '오른다.':'올라요.',
 '이어진다.':'이어져요.',
 '모아진다.':'모아져요.',
 '모은다.':'모아요.',
 '전해진다.':'전해져요.',
 '이루어진다.':'이뤄져요.',
 '올린다.':'올려요.',
 '노린다.':'노려요.',
 '다룬다.':'다뤄요.',
 '알린다.':'알려요.',
 '이른다.':'일러요.',
 '들어선다.':'들어서요.',
 '가한다.':'가해요.',
 '들려준다.':'들려줘요.',
 '여겨진다.':'여겨져요.',
 '나눈다.':'나눠요.',
 '떠난다.':'떠나요.',
 '나타난다.':'나타나요.',
 '알려준다.':'알려줘요.',
 '늘어난다.':'늘어나요.',
 '찾아간다.':'찾아가요.',
 '만난다.':'만나요.',
 '바뀐다.':'바뀌어요.',
 '높인다.':'높여요.',
 '꾸며진다.':'꾸며져요.',
 '맞춘다.':'맞춰요.',
 '꾀한다.':'꾀해요.',
 '파헤친다.':'파헤쳐요.',
 '그려진다.':'그려져요.',
 '나간다.':'나가요.',
 '밝힌다.':'밝혀요.',
 '치러진다.':'치러져요.',
 '알아본다.':'알아보아요.',
 '생긴다.':'생겨요.',
 '진다.':'져요.',
 '앞장선다.':'앞장서요.',
 '탄다.':'타요.',
 '낸다.':'내요.',
 '모인다.':'모여요.',
 '다.':'요.',
 '합니다.':'해요.',
 '겨룬다.':'겨뤄요.',
 '늘린다.':'늘려요.',
 '내린다.':'내려요.',
 '건다.':'걸어요.',
 '살펴본다.':'살펴보아요.',
 '치룬다.':'치뤄요.',
 '일으킨다.':'일으켜요.',
 '드러난다.':'드러나요.',
 '돌아온다.':'돌아와요.',
 '띈다.':'띄어요.',
 '쓰인다.':'쓰여요.',
 '요한다.':'필요로 해요.',
 '선다.':'서요.',
 '드러낸다.':'드러내요.',
 '불린다.':'불려요.',
 '든다.':'들어요.',
 '내려진다.':'내려져요.',
 '기울인다.':'기울여요.',
 '손꼽힌다.':'손꼽혀요.',
 '밝혀진다.':'밝혀져요.',
 '찾아온다.':'찾아와요.',
 '가린다.':'가려요.',
 '느껴진다.':'느껴져요.',
 '담아낸다.':'담아내요.',
 '알려진다.':'알려져요.',
 '그려낸다.':'그려내요.',
 '지닌다.':'지녀요.',
 '돋보인다.':'돋보여요.',
 '느낀다.':'느껴요.',
 '거듭난다.':'거듭나요.',
 '정해진다.':'정해져요.',
 '보낸다.':'보내요.',
 '엿보인다.':'엿보여요.',
 '다진다.':'다져요.',
 '다툰다.':'다퉈요.',
 '세운다.':'세워요.',
 '나타낸다.':'나타내요.',
 '치뤄진다.':'치뤄져요.',
 '불러일으킨다.':'불러일으켜요.',
 '점쳐진다.':'점쳐져요.',
 '이룬다.':'이뤄요.',
 '따른다.':'따라요.',
 '거친다.':'거쳐요.',
 '넓힌다.':'넓혀요.',
 '나뉜다.':'나뉘어요.',
 '바꾼다.':'바꿔요.',
 '다가온다.':'다가와요.',
 '달라진다.':'달라져요.',
 '꾸민다.':'꾸며요.',
 '마친다.':'마쳐요.',
 '바랍니다.':'바라요.',
 '들린다.':'들려요.',
 '인다.':'일어요.',
 '처해진다.':'처해져요.',
 '바라본다.':'바라봐요.',
 '엇갈린다.':'엇갈려요.',
 '던진다.':'던져요.',
 '채워진다.':'채워져요.',
 '달린다.':'달려요.',
 '남긴다.':'남겨요.',
 '풀어낸다.':'풀어내요.',
 '그린다.':'그려요.',
 '맡긴다.':'맡겨요.',
 '세워진다.':'세워져요.',
 '누린다.':'누려요.',
 '사라진다.':'사라져요.',
 '꾸린다.':'꾸려요.',
 '모른다.':'몰라요.',
 '돌아간다.':'돌아가요.',
 '낮춘다.':'낮춰요.',
 '보탠다.':'보태요.',
 '간다.':'가요.',
 '읽힌다.':'읽혀요.',
 '뗀다.':'떼요.',
 '통한다.':'통해요.',
 '정한다.':'정해요.',
 '기한다.':'기해요.',
 '됩니다.':'돼요.',
 '취한다.':'취해요.',
 '즐긴다.':'즐겨요.',
 '시킨다.':'시켜요.',
 '물러난다.':'물러나요.',
 '파헤쳐본다.':'파헤쳐보아요.',
 '다가간다.':'다가가요.',
 '찾아본다.':'찾아보아요.',
 '맞댄다.':'맞대요.',
 '달랜다.':'달래요.',
 '쓰여진다.':'쓰여져요.',
 '실린다.':'실려요.',
 '원한다.':'원해요.',
 '키운다.':'키워요.',
 '임한다.':'임해요.',
 '들어온다.':'들어와요.',
 '떠오른다.':'떠올라요.',
 '쓴다.':'써요.',
 '불태운다.':'불태워요.',
 '되돌아본다.':'뒤돌아보아요.',
 '들려온다.':'들려와요.',
 '푼다.':'풀어요.',
 '내세운다.':'내세워요.',
 '줄인다.':'줄여요.',
 '들여다본다.':'들여다보아요.',
 '내보낸다.':'내보내요.',
 '올라간다.':'올라가요.',
 '태어난다.':'태어나요.',
 '날린다.':'날려요.',
 '꾸려진다.':'꾸려져요.',
 '밝혀다.':'밝혀요.',
 '다뤄진다.':'다뤄져요.',
 '둘러본다.':'둘러보아요.',
 '돌려준다.':'돌려줘요.',
 '뛴다.':'뛰어요.',
 '입힌다.':'입혀요.',
 '해본다.':'해보아요.',
 '둔다.':'두어요.',
 '어려워진다.':'어려워져요.',
 '칭한다.':'불러요.',
 '끌어올린다.':'끌어올려요.',
 '자이다.':'자예요.',
 '부른다.':'불러요.',
 '논한다.':'논해요.',
 '처한다.':'처해요.',
 '가져온다.':'가져와요.',
 '그친다.':'그쳐요.',
 '미친다.':'미쳐요.',
 '두드러진다.':'두드러져요.',
 '들인다.':'들여요.',
 '지낸다.':'지내요.',
 '입니다.':'이고요.',
 '기다린다.':'기다려요.',
 '노려본다.':'노려보아요.',
 '팔린다.':'팔려요.',
 '벌어진다.':'벌어져요.',
 '살핀다.':'살펴요.',
 '뜬다.':'떠요.',
 '커진다.':'커져요.',
 '온다.':'와요.',
 '단다.':'달아요.',
 '줄어든다.':'줄어들어요.',
 '드높인다.':'드높여요.',
 '올려진다.':'올려져요.',
 '행한다.':'행해요.',
 '새긴다.':'새겨요.',
 '썻다.':'썼어요.',
 '깊어진다.':'깊어져요.',
 '지킨다.':'지켜요.',
 '마세요.':'말아요.',
 '퍼진다.':'퍼져요.',
 '잇따른다.':'연속되요.',
 '끼친다.':'줘요.',
 '행해진다.':'행해져요.',
 '찾아낸다.':'찾아내요.',
 '뿌린다.':'뿌려요.',
 '향한다.':'향해요.',
 '넘쳐난다.':'넘쳐나요.',
 '치운다.':'치워요.',
 '꺼린다.':'꺼려요.',
 '가려낸다.':'가려내요.',
 '저지른다.':'저질러요.',
 '평한다.':'평해요.',
 '꾸려간다.':'꾸려가요.',
 '앞선다.':'앞서요.',
 '깨운다.':'깨워요.',
 '지난다.':'지나요.',
 '칭해진다.':'불려와요.',
 '꾼다.':'꿔요.',
 '담근다.':'담가요.',
 '멈춘다.':'멈춰요.',
 '걸어온다.':'걸어와요.',
 '가른다.':'길러요.',
 '곁들인다.':'곁들여요.',
 '내려간다.':'내려가요.',
 '흘러갑니다.':'흘러가요.',
 '어울린다.':'어울려요.',
 '떨어진다.':'떨어져요.',
 '발표.':'발표해요.',
 '담긴다.':'담겨요.',
 '키워낸다.':'키워내요.',
 '채운다.':'채워요.',
 '빠진다.':'빠져요.',
 '난다.':'나요..',
 '가까워진다.':'가까워져요.',
 '가려진다.':'가려져요.',
 '문의.':'문의해요.',
 '핀다.':'피어요.',
 '적신다.':'적셔요.',
 '구한다.':'구해요.',
 '부린다.':'부려요.',
 '익힌다.':'익혀요.',
 '놀란다.':'놀라요.',
 '드린다.':'드려요.',
 '분한다.':'분해요.',
 '훔친다.':'훔쳐요.',
 '커보인다.':'커보여요.',
 '쏟아진다.':'쏟아져요.',
 '띤다.':'띄어요.',
 '켠다.':'켜요.',
 '옮긴다.':'옮겨요.',
 '끌어들인다.':'끌어들여요.',
 '쉬워진다.':'쉬워져요.',
 '기다려진다.':'기다려져요.',
 '누빈다.':'누벼요.',
 '새겨진다.':'새겨져요.',
 '없앤다.':'없애요.',
 '해야겠다.':'해야겠어요.',
 '끝낸다.':'끝내요.',
 '배운다.':'배워요.',
 '걸러낸다.':'걸러내요.',
 '가르친다.':'가르쳐요.',
 '뽐낸다.':'뽐내요.',
 '됏다.':'됐어요.',
 '빨라진다.':'빨라져요.',
 '북적인다.':'북적여요.',
 '넘친다.':'넘쳐요.',
 '서두른다.':'서둘러요.',
 '가리킨다.':'가리켜요.',
 '부쳐진다.':'부쳐져요.',
 '애쓴다.':'애써요.',
 '넘어간다.':'넘어가요.',
 '일깨운다.':'일깨워요.',
 '보입니다.':'보여요.',
 '편다.':'펴요.',
 '생각난다.':'생각나요.',
 '흐른다.':'흘러요.',
 '울려퍼진다.':'울려펴져요.',
 '좁힌다.':'좁혀요.',
 '돌본다.':'돌봐요.',
 '움직인다.':'움직여요.',
 '매겨진다.':'매겨져요.',
 '짜여진다.':'짜여져요.',
 '챙긴다.':'챙겨요.',
 '납니다.':'납니다.',
 '흘러나온다.':'흘러나와요.',
 '적어진다.':'적어져요.',
 '겁니다.':'거예요.',
 '벗어난다.':'벗어나요.',
 '기다립니다.':'기다려요.',
 '돋운다.':'돋워요.',
 '잡아낸다.':'잡아내요.',
 '열립니다.':'열려요.',
 '펼쳐보인다.':'펼쳐보여요.',
 '건넨다.':'건네요.',
 '번다.':'벌어요.',
 '기른다.':'길러요.',
 '꼬인다.':'꼬여요.',
 '판다.':'팔아요.',
 '나타냅니다.':'나타내요.',
 '몰린다.':'몰려요.',
 '가집시다.':'가져봐요.',
 '깨진다.':'깨져요.',
 '담겨진다.':'담겨져요.',
 '풀린다.':'풀려요.',
 '걸린다.':'걸려요.',
 '어긋난다.':'어긋나요.',
 '핬다.':'했어요.',
 '일어난다.':'일어나요.',
 '그리워진다.':'그리워져요.',
 '찾아갑니다.':'찾아가요.',
 '산다.':'살아요.',
 '돌아다본다.':'돌아봐요.',
 '묻힌다.':'묻혀요.',
 '벌린다.':'벌려요.',
 '받는다.':'받아요.',
 '있다.':'있어요.'}

In [7]:
def filter_stop(sen):
    '''
    생성요약문에 적용하여 마지막 어미를 마침표로 통일합니다.
    '''
    if sen[-1] in ['다', '한다', '된다']:
        sen += '.'
        
    elif sen[-1] == ',':
        sen = sen.rstrip(','); sen += '.'
        
    elif sen[-1] != '.':
        sen = False
        
    return sen

def convert(sen):
    '''
    문장의 종결어미를 문어체에서 구어체로 변환합니다.
    '''
    irregular_morph = ''
    irregular_pos = ''
    
    #- 문장을 공백단위 리스트로 변환
    lst = sen.split() 
    
    if lst[-1] == '.':
        lst.pop(); lst[-1] += '.'

    #- 문장의 어미만 형태소 분석
    morphs = mecab.morphs(lst[-1]) 
    
    #- 문장의 어미만 품사 태깅
    pos = [i[1] for i in mecab.pos(lst[-1])] 

    eoyo = [('VA','EF'),('VV','EF'),('VX','EF'),('EP','EF'),('VV+EP','EF'),('VX+EP','EF'),('XSV+EP','EF'),('XSA+EP','EF')]
        
    if len(pos) >= 3:
        #- 규칙 종결어미
        tmp = (pos[-3],pos[-2])       
        if tmp in eoyo:
            morphs[-2] = '어요'
        elif pos[-2] == 'VCP+EF':
            morphs[-2] = '예요'
        elif tmp == ('XSA','EF'):
            morphs[-3] = '해요'
            morphs[-2] = ''
        elif tmp == ('NNG','XSV+EF'):
            morphs[-2] = '해요'
        elif tmp == ('VCP','EF'):
            morphs[-2] = '에요'
        elif pos[-3][1] == 'EP':
            morphs[-4] = '했어요'
            morphs[-3] = ''
            morphs[-2] = '' 
        converted = ''.join(morphs)
        lst[-1] = converted
        
        
    else:
        #- 불규칙 종결어미
        if lst[-1] in convert_dict.keys():
            lst[-1] = convert_dict[lst[-1]]
        
#         irregular_morph = lst[-1]
#         irregular_pos = pos
    
    sen = ' '.join(lst)
#     return (sen, irregular_morph, irregular_pos)

    return sen



In [8]:
#- 마침표가 없는 생성요약문의 75개 제거
df['summary'] = df['summary'].map(lambda x: filter_stop(x))
df = df[df['summary'] != False]
#- 영문 데이터 4개 제거
df.drop([174856, 179922, 247382, 247963], inplace=True)

In [9]:
# df['irregular_morph_abs'] = df['summary'].map(lambda x: convert(x)[1])
# df['irregular_pos_abs'] = df['summary'].map(lambda x: convert(x)[2])
# list(df['irregular_morph_abs'].value_counts().index)

#- 영문 기사 제거
# tmp = ['providers.',  'increased.', 'deferred.',  'dust.']
# df[df['irregular_morph_abs'].map(lambda x: x in tmp)].index

In [10]:
%%time

#- 생성요약문 종결어미 변환
df['converted_abs'] = df['summary'].map(lambda x: convert(x))
#- 추출요약문 종결어미 변환
df['converted_ext_lst'] = df['ext_sum_lst'].map(lambda x: [convert(i) for i in x])
df['converted_ext'] = df['converted_ext_lst'].map(lambda x: ' '.join(x))


CPU times: user 31.7 s, sys: 282 ms, total: 32 s
Wall time: 32 s


In [11]:
list(df['original_text'].head(2))

['전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다. 농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha(3000평)당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 등 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는 논 타작물 지원사업 추진협의회 를 구성, 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 "최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다"며"쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다"고 말했다.',
 '여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의 도시를 완성했다. 시에 따르면 총 사업비 8억원이 투입된 컬러빌리지 사업은 낮에는 색채, 밤에는 빛 을 주제로 지난해 초 착공에 들어갔다. 컬러빌리지는 색채를 이용한 경관개선사업으로 사업완료

In [12]:
list(df['converted_ext'].head(5))

['전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했어요. 쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도예요. 논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 해요.',
 '여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의 도시를 완성했어요. 컬러빌리지는 색채를 이용한 경관개선사업으로 사업완료에 따라 고소천사벽화마을과 자산마을은 알록달록 색깔 옷을 입었어요. 사업 시행과 준공 과정에서도 주민들의 참여를 유도해 민관이 함께 만들어가는 경관사업의 좋은 사례를 만들었어요.',
 '임직원과 선수단 모두는 이날 구봉산 정상에 올라 일출을 보며 2018년 구단 목표를 달성하기 위한 결연한 의지를 다졌어요. 신승재 사장은"유상철 감독을 비롯한 코칭스텝, 선수단 구성이 마무리 된 만큼 구성원 모두가 하나되어 K리그 클래식 5위 이내 진입, FA컵 우승 등 ACL 진출권 획득을 목표로 최선을 다하자"고 선수들에게 신년 인사말을 전했어요. 한편 전남은 선수들도 추가 영입했어요.',
 '광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 1050명을 대상으로 새해 농업인 실용교육을 실시해요. 이번 교육은 농산물 FTA 확대와 경기침체 등 국내외 농업여건의 어려워짐에 따라 농업인들의 경쟁력을 높이고, 소득안정을 위해 마련됐어요. 교육은 시의 특화작목인 감, 참다래, 생강, 매실 등에 대한 저비용 생산과 고품질 재배기술을 중점적으로 다뤄요.',
 '전라남도는 올해 4월과 6월 홍콩 크루즈선사 스타크루즈의 5만톤급 아쿠아리우스 호가 타이완 관광객 4000여명을 싣고 대만 지룽( )항을 출발해 여수를 방문한다고 밝혔어요. 전라남도는 신규시장인 타이완 크루즈 유치를 위해 해양수산부와 공동으로 지난해 상반기부터 타이완에서 크루즈포트세일, 크루즈협회·여행사

In [13]:
list(df['converted_abs'].head(5))

["전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받어요.",
 '여수시는 컬러빌리지 사업에 8억원을 투입하여 색채와 빛 도시를 완성하여 고소천사벽화마을과 자산마을은 알록달록 색깔 옷을 입었고 사업 시행과 준공 과정에서도 주민들의 참여를 유도해 경관사업의 좋은 사례를 만들었어요.',
 '전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단 목표 달성을 위한 새해 각오를 다졌어요.',
 '광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작목 중심으로 농업인 실용교육을 실시해요.',
 '올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항해 전남의 관광지를 방문할 예정이에요.']

In [14]:
df = df[['title', 'original_text', 'converted_ext', 'converted_abs','size',
       'char_count', 'sum_char_cnt', 'ext_char_cnt', 'readable', 'accurate',
       'informative', 'trustworthy', 'media_sub_type']]

In [15]:
df.to_csv('aihub_convert.csv', index=False)